In [4]:
%reload_ext autoreload
%autoreload 2

import cv2
import numpy as np
import os
from aruco import detect_aruco, draw_aruco, select_aruco_poses, PoseSelectors
from segment_boxes import segment_and_draw_boxes_by_aruco, segment_boxes_by_color
from utils import show

In [5]:
calib = np.load('phitz/calib.npz')
K = calib['K']
D = calib['D']

dict_4x4 = cv2.aruco.Dictionary_get(cv2.aruco.DICT_4X4_1000)
params = cv2.aruco.DetectorParameters_create()
params.cornerRefinementMethod = cv2.aruco.CORNER_REFINE_APRILTAG
params.perspectiveRemovePixelPerCell = 8
params.perspectiveRemoveIgnoredMarginPerCell = 0.26

In [5]:
# side
folder = 'dataset/side/accepted/'
files = os.listdir(folder + 'aruco')
for f in files:
    image = cv2.imread(folder + 'aruco/' + f)
    box_image = cv2.imread(folder + 'box/' + f)
    arucos = detect_aruco(image, K=K, D=D, aruco_sizes=0.0295,
        use_generic=True, subtract=100, aruco_dict=dict_4x4, params=params)
    arucos = select_aruco_poses(arucos, PoseSelectors.Z_axis_up)
    draw = box_image.copy()
    segment_and_draw_boxes_by_aruco(draw, arucos, K, D)
    cv2.imwrite(folder + 'segmented/' + f, draw)

In [6]:
# top
folder = 'dataset/top/accepted/'
files = os.listdir(folder + 'aruco')
for f in files:
    image = cv2.imread(folder + 'aruco/' + f)
    box_image = cv2.imread(folder + 'box/' + f)
    arucos = detect_aruco(image, K=K, D=D, aruco_sizes=0.0295,
        use_generic=True, subtract=100, aruco_dict=dict_4x4, params=params)
    arucos = select_aruco_poses(arucos, PoseSelectors.Z_axis_back)
    draw = box_image.copy()
    segment_and_draw_boxes_by_aruco(draw, arucos, K, D)
    cv2.imwrite(folder + 'segmented/' + f, draw)

In [ ]:
folders = ["dataset/side/accepted/", "dataset/top/accepted/"]
for folder in folders:
    files = os.listdir(folder + "box/")
    for f in files:
        image_file = folder + "box/" + f
        image = cv2.imread(image_file)
        mask, _ = segment_boxes_by_color(image)
        polygons, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        draw = image.copy()
        cv2.polylines(draw, polygons, True, (255, 255, 255), thickness=1)
        overlay = draw.copy()
        for polygon in polygons:
            cv2.fillPoly(overlay, [polygon], (255, 255, 255))
        cv2.addWeighted(draw, 0.7, overlay, 0.3, 0, dst=draw)
        cv2.imwrite(folder + "masks/" + f, mask)
        cv2.imwrite(folder + "segmented_by_color/" + f, draw)

In [10]:
# test
image_file = 'dataset/top/accepted/aruco/0001.png'
image = cv2.imread(image_file)
arucos = detect_aruco(image, K=K, D=D, aruco_sizes=0.0295,
    use_generic=True, subtrack=100, aruco_dict=dict_4x4, params=params)
arucos = select_aruco_poses(arucos, PoseSelectors.worst)
draw = image.copy()
draw_aruco(draw, arucos, False, False, K, D)
show(draw)